# 다중회귀

## 음악 재생 수 예측하기
- 사용자의 음악 스트리밍 데이터를 기반으로 특정 곡의 재생 횟수를 예측하는 다중 회귀 모델을 만들고 평가해보자
- 제공 데이터
    - BPM (Beats Per Minute): 곡의 속도
    - Energy: 에너지 지표 (0~100)
    - Danceability: 춤출 수 있는 정도 (0~100)
    - Length: 곡의 길이 (초 단위)
    - Popularity: 곡의 인기 점수 (0~100)
    - Play_Count: 해당 곡의 총 재생 횟수 (목표 변수)

### 데이터 로드 및 탐색
- MusicStreaming.csv 데이터를 불러오고 기본적인 통계를 확인해보기
- 파일명 : high_popularity_spotify_data.csv
- 결측값이 있는지 확인하고, 필요하면 처리

In [37]:
import pandas as pd

df = pd.read_csv('../data/high_popularity_spotify_data.csv')
df[['track_popularity']]

,track_popularity
0,100
1,97
2,93
3,81
4,98
...,...
1681,76
1682,74
1683,69
1684,82


In [8]:
df.isnull().sum()

energy                      0
tempo                       0
danceability                0
playlist_genre              0
loudness                    0
liveness                    0
valence                     0
track_artist                0
time_signature              0
speechiness                 0
track_popularity            0
track_href                  0
uri                         0
track_album_name            1
playlist_name               0
analysis_url                0
track_id                    0
track_name                  0
track_album_release_date    0
instrumentalness            0
track_album_id              0
mode                        0
key                         0
duration_ms                 0
acousticness                0
id                          0
playlist_subgenre           0
type                        0
playlist_id                 0
dtype: int64

In [32]:
df = df.dropna()

### 데이터 전처리
- Genre를 원-핫 인코딩하여 숫자로 변환하기
- StandardScaler를 사용하여 BPM, Energy, Danceability, Length, Popularity를 정규화하기
- train_test_split을 사용하여 훈련 데이터와 테스트 데이터로 나누기

In [23]:
df[['playlist_genre']].value_counts()

playlist_genre
pop               357
rock              235
hip-hop           227
latin             184
electronic        148
gaming            100
ambient            61
arabic             50
r&b                50
punk               50
blues              45
metal              35
folk               33
afrobeats          20
brazilian          14
j-pop              11
classical          10
k-pop              10
indian              9
korean              8
turkish             7
reggae              5
world               4
indie               4
country             3
lofi                2
soul                2
jazz                1
Name: count, dtype: int64

In [61]:
from sklearn.model_selection import train_test_split
song_input = df[['tempo', 'energy', 'danceability', 'duration_ms']]
song_target = df[['track_popularity']]
train_input, test_input, train_target, test_target = train_test_split(song_input, song_target)

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

ss.fit(song_input)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

### 다중 회귀 모델 훈련
- `LinearRegression`을 사용하여 모델을 훈련시키기
- `track_popularity`를 예측하는 회귀 모델을 만들고, `R²` 점수를 평가하기

In [59]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input, test_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)

In [62]:
lr.fit(train_input, train_target)

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

-0.02342311999399116
-0.023840442373094684


C:\Users\1-04\Desktop\DAMF2\TIL\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\1-04\Desktop\DAMF2\TIL\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [107]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_poly, train_target)

print(lr.score(train_poly, train_target))
print(lr.score(test_poly, test_target))

0.01184769248123152
0.0038236058765693803


### 새로운 곡의 재생 수 예측
- 임의의 새로운 곡 데이터를 입력하고, 모델이 재생 횟수를 예측하도록 해보기

In [108]:
from sklearn.linear_model import SGDClassifier
sc = SGDClassifier(loss='log_loss', max_iter=100)
sc.fit(train_scaled, train_target)

print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

0.07278481012658228
0.05687203791469194


C:\Users\1-04\Desktop\DAMF2\TIL\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [109]:
sc.partial_fit(train_scaled, train_target)

print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

0.07436708860759493
0.06872037914691943


C:\Users\1-04\Desktop\DAMF2\TIL\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### 결과 시각화
- 예측된 track_popularity와 실제 값을 비교하는 **산점도(scatter plot)**를 그리기

In [ ]:
from matplotlib.pyplot as plt
plt.scatter(